<a href="https://colab.research.google.com/github/minerva79/medt_nlp/blob/main/MIMIC_IV_multiclass_classification_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Mon Feb 19 13:29:10 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla V100-SXM2-16GB           Off | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0              24W / 300W |      0MiB / 16384MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 54.8 gigabytes of available RAM

You are using a high-RAM runtime!


In [ ]:
import pandas as pd

# Load the pickle file into a pandas DataFrame
combined_df = pd.read_pickle("/content/drive/MyDrive/combined_df.pkl")

In [ ]:
import pandas as pd

# Assuming combined_df is your DataFrame and 'y' is the column with labels
combined_df = combined_df[combined_df['y'].apply(lambda x: len(x) <= 1)]

In [ ]:
from sklearn.model_selection import train_test_split

train_df, val_df = train_test_split(combined_df, test_size=0.2)


In [ ]:
import torch
from torch.utils.data import TensorDataset

# Create a mapping from class labels to integers
class_to_idx = {'anemia': 0, 'atrial fibrillation': 1, 'hyperlipidemia': 2, 'hypertension': 3, 'pneumonia': 4, 'no_label': 5}

# Function to convert class labels to integers
def get_class_idx(labels):
    if labels:  # If the list is not empty
        return class_to_idx[labels[0]]  # Assuming each list contains only one label
    else:
        return class_to_idx['no_label']  # Assign 'no_label' to empty lists

# Assuming combined_df is your full dataset and train_df, val_df are subsets of it
# Add 'class_idx' to combined_df
combined_df['class_idx'] = combined_df['y'].apply(get_class_idx)

# Ensure that train_df and val_df are updated with 'class_idx'
# This can be done in several ways depending on how train_df and val_df are created
# For example, if they are direct subsets, you can do:
train_df = combined_df.loc[train_df.index]
val_df = combined_df.loc[val_df.index]

# Now, your create_dataset function should work as expected
def create_dataset(df):
    input_ids = list(df['input_ids'])
    attention_masks = list(df['attention_mask'])
    labels = df['class_idx'].values  # Use the integer class labels

    return TensorDataset(torch.stack(input_ids), torch.stack(attention_masks), torch.tensor(labels, dtype=torch.long))

# Create datasets
train_dataset = create_dataset(train_df)
val_dataset = create_dataset(val_df)


In [ ]:
combined_df

,text,normalized_diseases,y,y_joined,processed_data,anemia,atrial fibrillation,hyperlipidemia,hypertension,pneumonia,input_ids,attention_mask,class_idx
0,name unit admission date discharge date date b...,[hypertension],[hypertension],hypertension,"([[101, 2171, 3131, 9634, 3058, 11889, 3058, 3...",0,0,0,1,0,"[tensor(0), tensor(13650), tensor(1933), tenso...","[tensor(1), tensor(1), tensor(1), tensor(1), t...",3
2,name unit admission date discharge date date b...,[anemia],[anemia],anemia,"([[101, 2171, 3131, 9634, 3058, 11889, 3058, 3...",1,0,0,0,0,"[tensor(0), tensor(13650), tensor(1933), tenso...","[tensor(1), tensor(1), tensor(1), tensor(1), t...",0
3,name unit admission date discharge date date b...,[atrial fibrillation],[atrial fibrillation],atrial fibrillation,"([[101, 2171, 3131, 9634, 3058, 11889, 3058, 3...",0,1,0,0,0,"[tensor(0), tensor(13650), tensor(1933), tenso...","[tensor(1), tensor(1), tensor(1), tensor(1), t...",1
7,name unit admission date discharge date date b...,[pneumonia],[pneumonia],pneumonia,"([[101, 2171, 3131, 9634, 3058, 11889, 3058, 3...",0,0,0,0,1,"[tensor(0), tensor(13650), tensor(1933), tenso...","[tensor(1), tensor(1), tensor(1), tensor(1), t...",4
8,name unit admission date discharge date date b...,[hyperlipidemia],[hyperlipidemia],hyperlipidemia,"([[101, 2171, 3131, 9634, 3058, 11889, 3058, 3...",0,0,1,0,0,"[tensor(0), tensor(13650), tensor(1933), tenso...","[tensor(1), tensor(1), tensor(1), tensor(1), t...",2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
11991,name unit admission date discharge date date b...,[hyperlipidemia],[hyperlipidemia],hyperlipidemia,"([[101, 2171, 3131, 9634, 3058, 11889, 3058, 3...",0,0,1,0,0,"[tensor(0), tensor(13650), tensor(1933), tenso...","[tensor(1), tensor(1), tensor(1), tensor(1), t...",2
11992,name unit admission date discharge date date b...,[],[],,"([[101, 2171, 3131, 9634, 3058, 11889, 3058, 3...",0,0,0,0,0,"[tensor(0), tensor(13650), tensor(1933), tenso...","[tensor(1), tensor(1), tensor(1), tensor(1), t...",5
11995,name unit admission date discharge date date b...,[hypertension],[hypertension],hypertension,"([[101, 2171, 3131, 9634, 3058, 11889, 3058, 3...",0,0,0,1,0,"[tensor(0), tensor(13650), tensor(1933), tenso...","[tensor(1), tensor(1), tensor(1), tensor(1), t...",3
11997,name unit admission date discharge date date b...,[],[],,"([[101, 2171, 3131, 9634, 3058, 11889, 3058, 3...",0,0,0,0,0,"[tensor(0), tensor(13650), tensor(1933), tenso...","[tensor(1), tensor(1), tensor(1), tensor(1), t...",5


In [ ]:
import torch
from torch.utils.data import DataLoader, TensorDataset

# Set the batch size
batch_size = 2  # You can adjust this according to your specific requirements

# Create the DataLoader for training data
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Create the DataLoader for validation data
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)


In [ ]:
from transformers import LongformerForSequenceClassification

model = LongformerForSequenceClassification.from_pretrained('allenai/longformer-base-4096', num_labels=5)
model.to(torch.device('cuda' if torch.cuda.is_available() else 'cpu'))


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/694 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/597M [00:00<?, ?B/s]

Some weights of LongformerForSequenceClassification were not initialized from the model checkpoint at allenai/longformer-base-4096 and are newly initialized: ['classifier.out_proj.weight', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


LongformerForSequenceClassification(
  (longformer): LongformerModel(
    (embeddings): LongformerEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
      (position_embeddings): Embedding(4098, 768, padding_idx=1)
    )
    (encoder): LongformerEncoder(
      (layer): ModuleList(
        (0-11): 12 x LongformerLayer(
          (attention): LongformerAttention(
            (self): LongformerSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (query_global): Linear(in_features=768, out_features=768, bias=True)
              (key_global): Linear(in_features=768, out_features=768, bias=True)
          

In [ ]:
# Set the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device: ", device)

Using device:  cuda


In [ ]:
from transformers import AdamW, get_linear_schedule_with_warmup
num_epochs = 3
optimizer = AdamW(model.parameters(), lr=1e-5)  # Learning rate
#optimizer = torch.optim.SGD(model.parameters(), lr=5e-6)

total_steps = len(train_dataloader) * num_epochs  # Number of training epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score
import torch
import numpy as np

def train(model, train_dataloader, validation_dataloader, optimizer, scheduler, accumulation_steps, num_epochs=3):
    for epoch_i in range(num_epochs):
        # Training
        model.train()
        total_train_loss = 0
        optimizer.zero_grad()

        for step, batch in enumerate(train_dataloader):
            b_input_ids = batch[0].to(device)
            b_input_mask = batch[1].to(device)
            b_labels = batch[2].to(device)

            outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
            loss = outputs.loss / accumulation_steps  # Normalize the loss since it is averaged
            total_train_loss += loss.item()

            loss.backward()

            # Perform optimization step every `accumulation_steps` batches
            if (step + 1) % accumulation_steps == 0 or (step + 1) == len(train_dataloader):
                optimizer.step()
                scheduler.step()
                optimizer.zero_grad()

        avg_train_loss = total_train_loss / len(train_dataloader)
        print(f"Epoch {epoch_i + 1} | Average Training Loss: {avg_train_loss}")

        # Validation
        model.eval()
        all_predictions = []
        all_true_labels = []

        for batch in validation_dataloader:
            b_input_ids = batch[0].to(device)
            b_input_mask = batch[1].to(device)
            b_labels = batch[2].to(device)

            with torch.no_grad():
                outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)

            logits = outputs.logits
            predictions = torch.argmax(logits, dim=1)

            # Move predictions and labels to CPU
            predictions = predictions.detach().cpu().numpy()
            label_ids = b_labels.to('cpu').numpy()

            # Store predictions and true labels
            all_predictions.append(predictions)
            all_true_labels.append(label_ids)

        # Convert list of arrays to single numpy arrays
        all_predictions = np.concatenate(all_predictions)
        all_true_labels = np.concatenate(all_true_labels)

        # Calculate evaluation metrics
        f1 = f1_score(all_true_labels, all_predictions, average='weighted')
        precision = precision_score(all_true_labels, all_predictions, average='weighted')
        recall = recall_score(all_true_labels, all_predictions, average='weighted')
        accuracy = accuracy_score(all_true_labels, all_predictions)

        print(f"Epoch {epoch_i + 1} | Validation F1-Score: {f1}")
        print(f"Epoch {epoch_i + 1} | Validation Precision: {precision}")
        print(f"Epoch {epoch_i + 1} | Validation Recall: {recall}")
        print(f"Epoch {epoch_i + 1} | Validation Accuracy: {accuracy}")

# Call the train function
train(model, train_dataloader, val_dataloader, optimizer, scheduler, accumulation_steps=3, num_epochs=1)


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
